In [66]:
import pandas as pd
from datetime import datetime
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [72]:
df = pd.read_csv('joe.csv')
df.columns = ['date', 'tvl', 'volume', 'fees(24hr)', 'revenue']
df['date'] = pd.to_datetime(df['date'])
df

,date,tvl,volume,fees(24hr),revenue
0,2021-08-08,2.705310e+07,NaN,NaN,NaN
1,2021-08-09,2.641678e+07,1.418957e+06,NaN,NaN
2,2021-08-10,2.505540e+07,1.326370e+06,NaN,NaN
3,2021-08-11,2.346859e+07,9.712907e+05,NaN,NaN
4,2021-08-12,2.313089e+07,7.371232e+05,NaN,NaN
5,2021-08-13,2.305233e+07,8.196791e+05,NaN,NaN
6,2021-08-14,2.291475e+07,5.995962e+05,NaN,NaN
7,2021-08-15,2.076003e+07,1.533653e+06,NaN,NaN
8,2021-08-16,2.434043e+07,3.723994e+06,NaN,NaN
9,2021-08-17,2.433395e+07,6.427245e+06,NaN,NaN


In [60]:

rows = []
for index, row in df.iterrows():
    date = row['date']
    data = cg.get_coin_history_by_id(id = 'joe', date = f"{date.day}-{date.month}-{date.year}")
    if 'market_data' in data:
        data2 = [date, row['tvl'], row['revenue'], data['id'], data['symbol'].upper(), data['name'], data['market_data']['current_price']['usd'], data['market_data']['market_cap']['usd'], data['market_data']['total_volume']['usd'], 0, 0, 0]
        rows.append(data2)
    else:
        print('Not enlisted Yet')

df2 = pd.DataFrame(rows, columns='date,tvl,revenue,id,symbol,name,price,mrk_cap,volume,max_supply,total_supply,circulating_supply'.split(','))


Not enlisted Yet


KeyboardInterrupt: 

In [ ]:
df2

In [73]:
import requests
import pandas as pd
from datetime import datetime
import pymysql
import numpy as np
import time
import traceback2
from pycoingecko import CoinGeckoAPI
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()
def insert_coingecko_data(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    df = df.astype(object).where(pd.notnull(df), None)
    for i, row in df.iterrows():
        dt = row["date"].timestamp()
        print(dt, row["symbol"], row["name"], row["price"], row["mrk_cap"], row["max_supply"], row["circulating_supply"], row["total_supply"], row['volume'], row['tvl'])
        try:
            sql = """INSERT INTO `coingecko_coin_mcap_supply_data` (`timestamp_dt`, `symbol`, `name`, `price`, `mrk_cap`,`max_supply`, `circulating_supply`, `total_supply`, `volume`, `tvl`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                `mrk_cap` = VALUES(mrk_cap),
                `price` = VALUES(price),
                `max_supply` = VALUES(max_supply),
                `circulating_supply` = VALUES(circulating_supply),
                `total_supply` = VALUES(total_supply),
                `volume` = VALUES(volume),
                `tvl` = VALUES(tvl)
                """
            cursor.execute(sql, (dt, row["symbol"], row["name"], row["price"], row["mrk_cap"], row["max_supply"], row["circulating_supply"], row["total_supply"], row['volume'], row['tvl']))
            # connection.commit()
        except Exception as e:
            print(e)
    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()
    cursor.close()
    connection.close()
    return

In [50]:
insert_coingecko_data(df2)

1628467200.0 JOE JOE 0.03451915928186395 0.0 0 0 0 132490.16011246745 26416781.78
1628553600.0 JOE JOE 0.03451915928186395 0.0 0 0 0 132490.16011246745 25055401.43
1628640000.0 JOE JOE 0.03292987756185703 0.0 0 0 0 108022.28843501415 23468591.3
1628726400.0 JOE JOE 0.02814990185979771 0.0 0 0 0 337297.1686786091 23130892.86
1628812800.0 JOE JOE 0.028902155233496155 0.0 0 0 0 100933.98248378551 23052328.38
1628899200.0 JOE JOE 0.029080232890759217 0.0 0 0 0 87467.90358111872 22914748.34
1628985600.0 JOE JOE 0.03373640624174375 0.0 0 0 0 188103.49142520915 20760033.06
1629072000.0 JOE JOE 0.03469212486603159 0.0 0 0 0 99282.3521268376 24340431.69
1629158400.0 JOE JOE 0.03269812073910181 0.0 0 0 0 132087.82626353038 24333953.07
1629244800.0 JOE JOE 0.03659673487758139 0.0 0 0 0 370884.3560649 26735466.84
1629331200.0 JOE JOE 0.06456069196645291 0.0 0 0 0 1038512.6770202812 39442373.31
1629417600.0 JOE JOE 0.08350036648429039 0.0 0 0 0 1830644.0730159506 55863176.13
1629504000.0 JOE JOE 0.

In [74]:
def insert_coin_metrics_df(df):
    df = df.astype(object).where(pd.notnull(df), None)
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        dt = row["date"].timestamp()
        try:
            print([row["date"], row["id"], row["name"], row["symbol"], row["mrk_cap"], row["volume"], row["ps"], row["tvl"], row["gmv"], row["revenue_supply_side"], row["revenue_protocol"], row["revenue"], row["price"]])
            sql = """INSERT INTO `tt_coin_metrics_v2` (`timestamp_dt`, `id`, `name`, `symbol`, `mrk_cap`, `volume`, `ps`, `tvl`, `gmv`, `revenue_supply_side`, `revenue_protocol`, `revenue`, `price`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
            `mrk_cap` = VALUES(mrk_cap),
            `volume` = VALUES(volume),
            `ps` = VALUES(ps),
            `tvl` = VALUES(tvl),
            `gmv` = VALUES(gmv),
            `revenue` = VALUES(revenue),
            `revenue_supply_side` = VALUES(revenue_supply_side),
            `revenue_protocol` = VALUES(revenue_protocol),
            `price` = VALUES(price)
            """
            cursor.execute(sql, (dt, row["id"], row["name"], row["symbol"], row["mrk_cap"], row["volume"], row["ps"], row["tvl"], row["gmv"], row["revenue_supply_side"], row["revenue_protocol"], row["revenue"], row["price"]))
#             connection.commit()
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()

In [75]:
df['id'] = 'joe'
df['name'] = 'JOE'
df['symbol'] = 'JOE'
df['mrk_cap'] = 0
df['mc_circulating'] = 0
df['market_cap_fully_diluted'] = 0
df['vol_mc'] = 0
df['ps'] = 0
df['tvl'] = 0
df['gmv'] = df['volume']
df['revenue_supply_side'] = 0
df['revenue_protocol'] = 0
df['price'] = 0
df


,date,tvl,volume,fees(24hr),revenue,id,name,symbol,mrk_cap,mc_circulating,market_cap_fully_diluted,vol_mc,ps,gmv,revenue_supply_side,revenue_protocol,price
0,2021-08-08,0,NaN,NaN,NaN,joe,JOE,JOE,0,0,0,0,0,NaN,0,0,0
1,2021-08-09,0,1.418957e+06,NaN,NaN,joe,JOE,JOE,0,0,0,0,0,1.418957e+06,0,0,0
2,2021-08-10,0,1.326370e+06,NaN,NaN,joe,JOE,JOE,0,0,0,0,0,1.326370e+06,0,0,0
3,2021-08-11,0,9.712907e+05,NaN,NaN,joe,JOE,JOE,0,0,0,0,0,9.712907e+05,0,0,0
4,2021-08-12,0,7.371232e+05,NaN,NaN,joe,JOE,JOE,0,0,0,0,0,7.371232e+05,0,0,0
5,2021-08-13,0,8.196791e+05,NaN,NaN,joe,JOE,JOE,0,0,0,0,0,8.196791e+05,0,0,0
6,2021-08-14,0,5.995962e+05,NaN,NaN,joe,JOE,JOE,0,0,0,0,0,5.995962e+05,0,0,0
7,2021-08-15,0,1.533653e+06,NaN,NaN,joe,JOE,JOE,0,0,0,0,0,1.533653e+06,0,0,0
8,2021-08-16,0,3.723994e+06,NaN,NaN,joe,JOE,JOE,0,0,0,0,0,3.723994e+06,0,0,0
9,2021-08-17,0,6.427245e+06,NaN,NaN,joe,JOE,JOE,0,0,0,0,0,6.427245e+06,0,0,0


In [ ]:
['id', 'name', 'symbol', 'date', 'mrk_cap', 'mc_circulating', 'market_cap_fully_diluted', 'volume', 'vol_mc', 'ps', 'tvl', 'gmv', 'revenue_supply_side', 'revenue_protocol', 'revenue', 'price']


In [76]:
insert_coin_metrics_df(df)

[Timestamp('2021-08-08 00:00:00'), 'joe', 'JOE', 'JOE', 0, None, 0, 0, None, 0, 0, None, 0]
[Timestamp('2021-08-09 00:00:00'), 'joe', 'JOE', 'JOE', 0, 1418956.68, 0, 0, 1418956.68, 0, 0, None, 0]
[Timestamp('2021-08-10 00:00:00'), 'joe', 'JOE', 'JOE', 0, 1326369.61, 0, 0, 1326369.61, 0, 0, None, 0]
[Timestamp('2021-08-11 00:00:00'), 'joe', 'JOE', 'JOE', 0, 971290.74, 0, 0, 971290.74, 0, 0, None, 0]
[Timestamp('2021-08-12 00:00:00'), 'joe', 'JOE', 'JOE', 0, 737123.18, 0, 0, 737123.18, 0, 0, None, 0]
[Timestamp('2021-08-13 00:00:00'), 'joe', 'JOE', 'JOE', 0, 819679.08, 0, 0, 819679.08, 0, 0, None, 0]
[Timestamp('2021-08-14 00:00:00'), 'joe', 'JOE', 'JOE', 0, 599596.18, 0, 0, 599596.18, 0, 0, None, 0]
[Timestamp('2021-08-15 00:00:00'), 'joe', 'JOE', 'JOE', 0, 1533653.16, 0, 0, 1533653.16, 0, 0, None, 0]
[Timestamp('2021-08-16 00:00:00'), 'joe', 'JOE', 'JOE', 0, 3723993.98, 0, 0, 3723993.98, 0, 0, None, 0]
[Timestamp('2021-08-17 00:00:00'), 'joe', 'JOE', 'JOE', 0, 6427245.27, 0, 0, 6427245